<a href="https://colab.research.google.com/github/JoRanjit/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/{spark_version}/{spark_version}-bin-hadoop2.7.tgz
!tar xf {spark_version}-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-16 02:16:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-08-16 02:16:35 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

**Vine Table Analysis**

In [ ]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"]).drop_duplicates()
vine_df.show(5,truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2JH1KG23SL3BW|4          |0            |0          |N   |Y                |
|R2D09ANI67HRV4|5          |0            |0          |N   |Y                |
|R32COH1DZMMBJG|5          |0            |0          |N   |Y                |
|RETW3NJ87V49W |5          |0            |0          |N   |Y                |
|R2XZUHX2RTNLWS|4          |0            |0          |N   |Y                |
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
#retrieve all the rows where the total_votes count is equal to or greater than 20 
#count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on
total_votes_df = vine_df.filter(vine_df["total_votes"] >= 20)
total_votes_df.show(5,truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1W4ITP3WWGU5H|1          |64           |77         |N   |Y                |
|RRS50ZY3B8Z24 |5          |24           |24         |N   |Y                |
|RGHBDOV2XGF5K |5          |60           |63         |N   |Y                |
|R6Q3DQDE62XCL |5          |33           |33         |N   |Y                |
|R3J9VFEGUJQUSX|5          |22           |23         |N   |Y                |
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
#retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter((total_votes_df["helpful_votes"]/total_votes_df["total_votes"]) >= 0.50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1W4ITP3WWGU5H|          1|           64|         77|   N|                Y|
| RRS50ZY3B8Z24|          5|           24|         24|   N|                Y|
| RGHBDOV2XGF5K|          5|           60|         63|   N|                Y|
| R6Q3DQDE62XCL|          5|           33|         33|   N|                Y|
|R3J9VFEGUJQUSX|          5|           22|         23|   N|                Y|
|R2SRSGDA3HYMS6|          5|           44|         45|   N|                Y|
|R1UHE5UT4IURRV|          1|           20|         20|   N|                Y|
| RHW1BPAH18W44|          4|           21|         22|   N|                Y|
|R1GKPJ0OBWVK6B|          5|           56|         59|   N|                Y|
| RYA0Y970C626N|          5|           19|         22|   N|     

In [ ]:
from pyspark.sql.functions import col
#retrieves all the rows where a review was written as part of the Vine program (paid)
paid_reviews_df = helpful_votes_df.filter(col("vine") == 'Y')
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1K4X0HJM0H7HX|          4|           32|         33|   Y|                N|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|
|R1DB74PJOC27NS|          4|          286|        321|   Y|                N|
| RI4QLSZJRLB43|          5|           91|        110|   Y|                N|
| RX3MGE1863YUZ|          4|           31|         32|   Y|                N|
|R1VJNLH762PTGT|          5|           20|         22|   Y|                N|
| RTF6DSZ1UTLHH|          5|          401|        418|   Y|                N|
|R1GZ0OFP9ZO9KB|          5|           23|         29|   Y|                N|
|R1ZP3IQ3VCZG2H|          5|           52|         55|   Y|                N|
| RGNC5HFA9IZ5V|          5|           59|         63|   Y|     

In [ ]:
#retrieves all the rows where a review was not part of the Vine program (unpaid)
unpaid_reviews_df = helpful_votes_df.filter(col("vine") == 'N')
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1W4ITP3WWGU5H|          1|           64|         77|   N|                Y|
| RRS50ZY3B8Z24|          5|           24|         24|   N|                Y|
| RGHBDOV2XGF5K|          5|           60|         63|   N|                Y|
| R6Q3DQDE62XCL|          5|           33|         33|   N|                Y|
|R3J9VFEGUJQUSX|          5|           22|         23|   N|                Y|
|R2SRSGDA3HYMS6|          5|           44|         45|   N|                Y|
|R1UHE5UT4IURRV|          1|           20|         20|   N|                Y|
| RHW1BPAH18W44|          4|           21|         22|   N|                Y|
|R1GKPJ0OBWVK6B|          5|           56|         59|   N|                Y|
| RYA0Y970C626N|          5|           19|         22|   N|     

In [ ]:
#five-star reviews
five_star_df = helpful_votes_df.filter(col("star_rating") == '5')
five_star_count = five_star_df.count()
five_star_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RRS50ZY3B8Z24|          5|           24|         24|   N|                Y|
| RGHBDOV2XGF5K|          5|           60|         63|   N|                Y|
| R6Q3DQDE62XCL|          5|           33|         33|   N|                Y|
|R3J9VFEGUJQUSX|          5|           22|         23|   N|                Y|
|R2SRSGDA3HYMS6|          5|           44|         45|   N|                Y|
|R1GKPJ0OBWVK6B|          5|           56|         59|   N|                Y|
| RYA0Y970C626N|          5|           19|         22|   N|                Y|
|R3FV3TKMFHDAZZ|          5|           32|         36|   N|                Y|
|R3EKKI8JSY2AOY|          5|           23|         23|   N|                Y|
|R336OJ547AP7L5|          5|           38|         41|   N|     

In [ ]:
#paid five star reviews
paid_five_star_df = paid_reviews_df.filter(col("star_rating") == '5')
paid_five_star_count = paid_five_star_df.count()
paid_five_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|
| RI4QLSZJRLB43|          5|           91|        110|   Y|                N|
|R1VJNLH762PTGT|          5|           20|         22|   Y|                N|
| RTF6DSZ1UTLHH|          5|          401|        418|   Y|                N|
|R1GZ0OFP9ZO9KB|          5|           23|         29|   Y|                N|
|R1ZP3IQ3VCZG2H|          5|           52|         55|   Y|                N|
| RGNC5HFA9IZ5V|          5|           59|         63|   Y|                N|
|R1PA7HPH6FM9X2|          5|           32|         36|   Y|                N|
|R1PVOPQUK5I315|          5|          119|        133|   Y|                N|
|R2XY64NYWXDG8U|          5|           39|         45|   Y|     

In [ ]:
#unpaid five star reviews
unpaid_five_star_df = unpaid_reviews_df.filter(col("star_rating") == '5')
unpaid_five_star_count = unpaid_five_star_df.count()
unpaid_five_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RRS50ZY3B8Z24|          5|           24|         24|   N|                Y|
| RGHBDOV2XGF5K|          5|           60|         63|   N|                Y|
| R6Q3DQDE62XCL|          5|           33|         33|   N|                Y|
|R3J9VFEGUJQUSX|          5|           22|         23|   N|                Y|
|R2SRSGDA3HYMS6|          5|           44|         45|   N|                Y|
|R1GKPJ0OBWVK6B|          5|           56|         59|   N|                Y|
| RYA0Y970C626N|          5|           19|         22|   N|                Y|
|R3FV3TKMFHDAZZ|          5|           32|         36|   N|                Y|
|R3EKKI8JSY2AOY|          5|           23|         23|   N|                Y|
|R336OJ547AP7L5|          5|           38|         41|   N|     

In [ ]:
#get count of total reviews 
helpful_votes_count = helpful_votes_df.count()
helpful_votes_count

18155

In [ ]:
#5 start reviews compared to total reviews
five_star_perc = five_star_count/helpful_votes_count
round(five_star_perc,2)

0.47

In [ ]:
#paid 5 star count perc
paid_five_star_perc = paid_five_star_count/five_star_count
round(paid_five_star_perc,2)

0.01

In [ ]:
#unpaid 5 star count perc
unpaid_five_star_perc = unpaid_five_star_count/five_star_count
round(unpaid_five_star_perc,2)

0.99

In [ ]:
print(f"Out of tht total {helpful_votes_count} helpful votes, {five_star_count} were Five Star Reviews.")
print(f"And out of this only {paid_five_star_count} were Paid reviews, i.e, {round(paid_five_star_perc,2)}%, and the rest {unpaid_five_star_count} were Unpaid, i.e, {round(unpaid_five_star_perc,2)}% votes.")


Out of tht total 18155 helpful votes, 8556 were Five Star Reviews.
And out of this only 74 were Paid reviews, i.e, 0.01%, and the rest 8482 were Unpaid, i.e, 0.99% votes.
